In [ ]:
import time
import multiprocessing as mp
import sys
import queue

def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

def find_highest_prime(start, end, result_queue, timeout):
    proc_name = mp.current_process().name
    try:
        # Minimal work to ensure it runs
        print(f"{proc_name} started: range {start} to {end}", flush=True)
        with open(f"{proc_name}.log", "a") as f:
            f.write(f"{proc_name} started: range {start} to {end}\n")
        
        highest_prime = 0
        start_time = time.time()
        for num in range(start, end):
            if time.time() - start_time > timeout:
                print(f"{proc_name} timed out", flush=True)
                with open(f"{proc_name}.log", "a") as f:
                    f.write(f"{proc_name} timed out\n")
                break
            if is_prime(num):
                highest_prime = num
        
        print(f"{proc_name} finished: highest prime = {highest_prime}", flush=True)
        with open(f"{proc_name}.log", "a") as f:
            f.write(f"{proc_name} finished: highest prime = {highest_prime}\n")
        result_queue.put(highest_prime)
    except Exception as e:
        print(f"{proc_name} error: {e}", flush=True)
        with open(f"{proc_name}.log", "a") as f:
            f.write(f"{proc_name} error: {e}\n")
        result_queue.put(0)

def multi_core_calc(timeout=180):
    print("Starting Multiprocessing calculation...", flush=True)
    overall_start = time.time()
    
    num_processes = 2
    result_queue = mp.Queue()
    chunk_size = 100_000
    processes = []
    
    print(f"Using {num_processes} processes, chunk size = {chunk_size}", flush=True)
    
    for i in range(num_processes):
        start = i * chunk_size
        end = start + chunk_size
        p = mp.Process(target=find_highest_prime, args=(start, end, result_queue, timeout/num_processes))
        processes.append(p)
        p.start()
        print(f"Started process {i} (PID {p.pid}) for range {start} to {end}", flush=True)
    
    highest_primes = []
    for i in range(num_processes):
        try:
            # Wait full timeout only once, then shorter for remaining
            wait_time = timeout if i == 0 else 10
            result = result_queue.get(timeout=wait_time)
            highest_primes.append(result)
            print(f"Got result from process {i}: {result}", flush=True)
        except queue.Empty:
            print(f"Timeout waiting for process {i} after {wait_time}s", flush=True)
            highest_primes.append(0)
        except Exception as e:
            print(f"Error getting result from process {i}: {e}", flush=True)
            highest_primes.append(0)
    
    for i, p in enumerate(processes):
        if p.is_alive():
            print(f"Process {i} (PID {p.pid}) still alive, terminating...", flush=True)
            p.terminate()
        p.join()
    
    highest_prime = max(highest_primes) if highest_primes else 0
    time_taken = time.time() - overall_start
    
    print(f"Multiprocessing: Highest prime = {highest_prime} in {time_taken:.2f}s", flush=True)
    return highest_prime

if __name__ == "__main__":
    try:
        timeout = 180
        result = multi_core_calc(timeout)
        print(f"Final result: {result}", flush=True)
    except Exception as e:
        print(f"Main execution error: {e}", flush=True)
        sys.exit(1)

Starting Multiprocessing calculation...
Using 2 processes, chunk size = 100000
Started process 0 (PID 17512) for range 0 to 100000
Started process 1 (PID 24112) for range 100000 to 200000
Timeout waiting for process 0 after 180s
Timeout waiting for process 1 after 10s
Multiprocessing: Highest prime = 0 in 190.09s
Final result: 0
